In [ ]:
import os
import numpy as np
import pandas as pd
import random
import torch
from autogluon.tabular import TabularDataset, TabularPredictor

def setup_seed(seed = 3407):
    os.environ['PYTHONHASHSEED'] = str(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

setup_seed()

train_data = TabularDataset('Path to traning cohort')

drop_colums = ['ID']
train_data = train_data.drop(columns=drop_colums)

label = 'Label'
metric = 'roc_auc'
# time_limit = 1200
save_path = 'Path to save your model'

# predictor = TabularPredictor(label=label, path=save_path, eval_metric = metric).fit(train_data, presets='best_quality', time_limit=time_limit)
predictor = TabularPredictor(label=label, path=save_path, eval_metric = metric).fit(train_data, presets='medium_quality')

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
test_data = TabularDataset('Path to testing cohort').drop(columns=drop_colums)

label = 'Label'
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
test_data_nolab.head()

In [ ]:
predictor = TabularPredictor.load(save_path)
y_pred_probe = predictor.predict_proba(test_data_nolab)
print("Predictions:  \n", y_pred_probe)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_probe, auxiliary_metrics=True)

In [ ]:
predictor.leaderboard(train_data, silent=True)

In [ ]:
predictor.feature_importance(train_data)

In [ ]:
predictor.leaderboard(test_data, silent=True)

In [ ]:
predictor.feature_importance(test_data)